In [1]:
# ============================================================
# OMEGA INFINITY v3 - META CADENA DE MARKOV (ARREGLADO)
# ============================================================

import numpy as np
from collections import defaultdict
import struct
import zlib

class MetaMarkovCompressor:
    """
    Meta Cadena de Markov - f(m) = m
    Versión corregida con codificación precisa
    """

    def __init__(self, order=3):
        self.order = order

    def compress(self, text):
        print(f"{'='*60}")
        print(f"🌌 OMEGA INFINITY v3 - META MARKOV")
        print(f"{'='*60}")
        print(f"📝 Texto original: {len(text)} caracteres")
        print(f"📊 Orden de Markov: {self.order}")
        print(f"{'='*60}\n")

        # Modelo adaptativo
        transitions = defaultdict(lambda: defaultdict(int))

        # Alfabeto ordenado
        alphabet = sorted(set(text))
        char_to_idx = {c: i for i, c in enumerate(alphabet)}

        # Prefijo literal
        prefix = text[:self.order]

        # Generar ranks Y guardar tamaños de contexto
        ranks = []
        max_values = []  # Valor máximo posible para cada rank

        for i in range(self.order, len(text)):
            context = text[i-self.order:i]
            current_char = text[i]

            if context in transitions:
                # Ordenar caracteres vistos por frecuencia, luego alfabético
                seen_chars = sorted(
                    transitions[context].items(),
                    key=lambda x: (-x[1], x[0])
                )
                seen_list = [c for c, _ in seen_chars]

                if current_char in seen_list:
                    # Carácter conocido en contexto
                    rank = seen_list.index(current_char)
                    max_val = len(seen_list)  # 0 to len-1 para conocidos, len para escape
                else:
                    # Escape: carácter nuevo en este contexto
                    rank = len(seen_list)  # Código de escape
                    max_val = len(seen_list) + 1

                ranks.append(rank)
                max_values.append(max_val)

                # Si es escape, también necesitamos el índice en alfabeto
                if current_char not in seen_list:
                    # Caracteres NO vistos en este contexto
                    unseen = [c for c in alphabet if c not in seen_list]
                    unseen_idx = unseen.index(current_char)
                    ranks.append(unseen_idx)
                    max_values.append(len(unseen))
            else:
                # Contexto completamente nuevo: usar alfabeto
                rank = char_to_idx[current_char]
                ranks.append(rank)
                max_values.append(len(alphabet))

            # Actualizar modelo
            transitions[context][current_char] += 1

        # Empaquetar ranks con sus max_values
        packed = self._pack_ranks(ranks, max_values)

        # Construir paquete
        package = bytearray()

        # Header
        package.extend(struct.pack('B', self.order))
        package.extend(struct.pack('I', len(text)))

        # Alfabeto
        alphabet_str = ''.join(alphabet)
        alphabet_bytes = alphabet_str.encode('utf-8')
        package.extend(struct.pack('H', len(alphabet_bytes)))
        package.extend(alphabet_bytes)

        # Prefijo
        prefix_bytes = prefix.encode('utf-8')
        package.extend(struct.pack('B', len(prefix_bytes)))
        package.extend(prefix_bytes)

        # Número de ranks
        package.extend(struct.pack('I', len(ranks)))

        # Max values (necesarios para decodificar)
        for mv in max_values:
            if mv < 256:
                package.append(mv)
            else:
                package.append(255)
                package.extend(struct.pack('H', mv))

        # Ranks empaquetados
        package.extend(packed)

        # Compresión final
        compressed = zlib.compress(bytes(package), level=9)

        # Stats
        original_size = len(text.encode('utf-8'))
        compressed_size = len(compressed)
        ratio = (1 - compressed_size / original_size) * 100

        print(f"📦 Ranks: {len(ranks)}")
        print(f"📉 Max rank: {max(ranks) if ranks else 0}")
        print(f"📊 Promedio: {np.mean(ranks):.2f}" if ranks else "N/A")
        print(f"\n{'='*60}")
        print(f"📁 Original: {original_size} bytes")
        print(f"📦 Comprimido: {compressed_size} bytes")
        print(f"📉 Ratio: {ratio:.2f}%")
        print(f"🔢 Factor: {original_size/compressed_size:.2f}x")
        print(f"{'='*60}")

        return compressed

    def _pack_ranks(self, ranks, max_values):
        """Empaqueta ranks usando el número óptimo de bits"""
        bits = []

        for rank, max_val in zip(ranks, max_values):
            if max_val <= 1:
                # Solo un valor posible, 0 bits necesarios
                continue

            num_bits = int(np.ceil(np.log2(max_val + 1)))

            for b in range(num_bits - 1, -1, -1):
                bits.append((rank >> b) & 1)

        # Padding a bytes
        while len(bits) % 8 != 0:
            bits.append(0)

        # Convertir a bytes
        result = bytearray()
        for i in range(0, len(bits), 8):
            byte = 0
            for j in range(8):
                byte = (byte << 1) | bits[i + j]
            result.append(byte)

        return bytes(result)

    def decompress(self, compressed):
        print(f"\n{'='*60}")
        print(f"🔓 DESCOMPRESIÓN")
        print(f"{'='*60}")

        data = zlib.decompress(compressed)
        pos = 0

        # Header
        order = struct.unpack('B', data[pos:pos+1])[0]; pos += 1
        original_len = struct.unpack('I', data[pos:pos+4])[0]; pos += 4

        # Alfabeto
        alphabet_len = struct.unpack('H', data[pos:pos+2])[0]; pos += 2
        alphabet = list(data[pos:pos+alphabet_len].decode('utf-8')); pos += alphabet_len

        # Prefijo
        prefix_len = struct.unpack('B', data[pos:pos+1])[0]; pos += 1
        prefix = data[pos:pos+prefix_len].decode('utf-8'); pos += prefix_len

        # Número de ranks
        num_ranks = struct.unpack('I', data[pos:pos+4])[0]; pos += 4

        # Max values
        max_values = []
        for _ in range(num_ranks):
            mv = data[pos]; pos += 1
            if mv == 255:
                mv = struct.unpack('H', data[pos:pos+2])[0]; pos += 2
            max_values.append(mv)

        # Bits de ranks
        bits_data = data[pos:]
        bits = []
        for byte in bits_data:
            for b in range(7, -1, -1):
                bits.append((byte >> b) & 1)

        # Decodificar ranks
        ranks = []
        bit_pos = 0

        for max_val in max_values:
            if max_val <= 1:
                ranks.append(0)
                continue

            num_bits = int(np.ceil(np.log2(max_val + 1)))
            rank = 0
            for _ in range(num_bits):
                rank = (rank << 1) | bits[bit_pos]
                bit_pos += 1
            ranks.append(rank)

        # Reconstruir texto
        result = list(prefix)
        transitions = defaultdict(lambda: defaultdict(int))
        char_to_idx = {c: i for i, c in enumerate(alphabet)}

        rank_idx = 0

        while len(result) < original_len and rank_idx < len(ranks):
            context = ''.join(result[-order:])

            if context in transitions:
                seen_chars = sorted(
                    transitions[context].items(),
                    key=lambda x: (-x[1], x[0])
                )
                seen_list = [c for c, _ in seen_chars]

                rank = ranks[rank_idx]
                rank_idx += 1

                if rank < len(seen_list):
                    # Carácter conocido
                    char = seen_list[rank]
                else:
                    # Escape: leer índice en no vistos
                    unseen = [c for c in alphabet if c not in seen_list]
                    unseen_idx = ranks[rank_idx]
                    rank_idx += 1
                    char = unseen[unseen_idx]
            else:
                # Contexto nuevo
                rank = ranks[rank_idx]
                rank_idx += 1
                char = alphabet[rank]

            result.append(char)
            transitions[context][char] += 1

        text = ''.join(result)
        print(f"✅ Recuperado: {len(text)} caracteres")
        print(f"{'='*60}")

        return text


# ============================================================
# PRUEBAS
# ============================================================

def test():
    textos = {
        "simple": "aaabbbcccaaabbbccc",
        "repetitivo": "abracadabra " * 50,
        "natural": """La inteligencia artificial es una rama de la informática
        que busca crear sistemas capaces de realizar tareas que normalmente
        requieren inteligencia humana.""" * 5,
        "código": "def f(n):\n    return n * 2\n" * 20,
        "binario": "01001011" * 100,
    }

    print("\n" + "🌌"*30)
    print("  OMEGA INFINITY v3 - CORREGIDO")
    print("🌌"*30 + "\n")

    resultados = []

    for nombre, texto in textos.items():
        print(f"\n{'#'*60}")
        print(f"# {nombre.upper()}")
        print(f"{'#'*60}")

        comp = MetaMarkovCompressor(order=3)

        try:
            compressed = comp.compress(texto)
            recovered = comp.decompress(compressed)

            match = texto == recovered
            print(f"\n🔍 Verificación: {'✅ CORRECTO' if match else '❌ ERROR'}")

            if not match:
                print(f"\n   Original length: {len(texto)}")
                print(f"   Recovered length: {len(recovered)}")
                for i in range(min(len(texto), len(recovered))):
                    if texto[i] != recovered[i]:
                        print(f"   Primera diferencia en pos {i}:")
                        print(f"   Original: ...{repr(texto[max(0,i-5):i+10])}...")
                        print(f"   Recovered: ...{repr(recovered[max(0,i-5):i+10])}...")
                        break

            original_size = len(texto.encode('utf-8'))
            compressed_size = len(compressed)
            resultados.append({
                'nombre': nombre,
                'original': original_size,
                'comprimido': compressed_size,
                'ratio': (1 - compressed_size/original_size) * 100,
                'match': match
            })

        except Exception as e:
            print(f"❌ Error: {e}")
            import traceback
            traceback.print_exc()

    # Resumen
    print(f"\n\n{'='*60}")
    print("📊 RESUMEN FINAL")
    print(f"{'='*60}")
    print(f"{'Texto':<15} {'Original':>10} {'Comprimido':>12} {'Ratio':>10} {'OK':>5}")
    print(f"{'-'*60}")
    for r in resultados:
        status = "✅" if r['match'] else "❌"
        print(f"{r['nombre']:<15} {r['original']:>10} {r['comprimido']:>12} {r['ratio']:>9.1f}% {status:>5}")

test()


🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌
  OMEGA INFINITY v3 - CORREGIDO
🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌🌌


############################################################
# SIMPLE
############################################################
🌌 OMEGA INFINITY v3 - META MARKOV
📝 Texto original: 18 caracteres
📊 Orden de Markov: 3

📦 Ranks: 15
📉 Max rank: 2
📊 Promedio: 0.60

📁 Original: 18 bytes
📦 Comprimido: 32 bytes
📉 Ratio: -77.78%
🔢 Factor: 0.56x

🔓 DESCOMPRESIÓN
✅ Recuperado: 18 caracteres

🔍 Verificación: ✅ CORRECTO

############################################################
# REPETITIVO
############################################################
🌌 OMEGA INFINITY v3 - META MARKOV
📝 Texto original: 600 caracteres
📊 Orden de Markov: 3

📦 Ranks: 598
📉 Max rank: 5
📊 Promedio: 0.21

📁 Original: 600 bytes
📦 Comprimido: 52 bytes
📉 Ratio: 91.33%
🔢 Factor: 11.54x

🔓 DESCOMPRESIÓN
✅ Recuperado: 600 caracteres

🔍 Verificación: ✅ CORRECTO

############################################################
# NATU